## Imports

In [ ]:
import logging
import os
from pathlib import Path

from __future__ import annotations

# from omero_quay import srce_path

from omero_quay.managers.rocrate import RoCrateManager
from omero_quay.core.config import get_conf
from omero_quay.parsers.excel import XlsxParser, parse_xlsx
from omero_quay.core.connect import omero_conn
from omero_quay.managers.rocrate import SRC_DIR, RoCrateManager

log = logging.getLogger(__name__)

srce_path = (SRC_DIR / "../..").resolve()
templates = SRC_DIR / "templates"
icons = SRC_DIR / "icons"
tests = (srce_path / "tests").resolve()
log = logging.getLogger("omero_quay")
log.setLevel("WARNING")

## Settings

### Configuration file path

In [ ]:
# path to the the configuration file
os.environ["QUAY_CONF"] = (tests / "containers/quay_docker_host.yml").as_posix()

print(
    f"Using configuration from {os.environ['QUAY_CONF']}",
)

In [ ]:
conf = get_conf()
# local path to the test data
# also sets the path to the default excel file

test_data_path = Path(os.environ.get("QUAY_TEST_DATA", ".")).resolve()

conf["quay"]["DATA_PATH"] = test_data_path
conf["ingest"][
    "PROVENANCE_URL"
] = f"file://{(tests / 'containers/provenance').resolve()}"


source_data_path = (test_data_path / "facility0" / "isa").resolve()
if source_data_path.exists():
    print(
        "Using data from ",
        source_data_path,
    )
else:
    print(f"{source_data_path} does not exist")

### Data path and parsing

In [ ]:
# from requests import NewConnectionError

# path to the excel file relative to `test_data_path`

xlsx_path = test_data_path / "excels" / "test_JCB_facility.xlsx"
if not xlsx_path.exists():
    print("Bad path")
else:
    print(f"Using {xlsx_path.resolve()}")

try:
    manifest = parse_xlsx(xlsx_path, conf)
except Exception as e:
    print(
        """
    You might not have set docker up before you ran this notebook.

    Refer to the previous section and start the containers in a terminal.
    """
    )

## Ro-Crate generation

In [ ]:
manifest.step = 0
# RoCrateManager is a clerk whose purpose is to generate rocrate content.
with RoCrateManager(conf, manifest, source_data_path) as rocrate_clerk:
    # ajust the log level if you need
    # detailed input (eg set to INFO)
    rocrate_clerk.log.setLevel("WARNING")
    print(rocrate_clerk.data_path)
    rocrate_clerk.routine()

    # A clerk has a nice printed representation
    # print(rocrate_clerk)
    rocrate_clerk.set_state("changed")

In [ ]:
!ls /home/guillaume/Dev/Omero/QuayTestData/facility0/isa/JCB2009

In [ ]:
import json

In [ ]:
rcj = "/home/guillaume/Dev/Omero/QuayTestData/facility0/isa/JCB2009/ro-crate-metadata.json"
with Path(rcj).open("r") as rcj:
    rocrate = json.loads(rcj.read())

The code works nice overall, but still a known problem:

- Vignette generation is not working, not because of a code problem per-se, but more because of a implementation problem in manifest generation (no raw image path implementation, which make the matching step failing).

Comments:

- Ro-Crate validation. The function works when you launch it next to dataset and rocrate (sent path is "./") but the subdirectories names must match the contents of the RoCrate file.

## How to use

First, create (or recreate) your containers:

```bash
make docker_purge
make docker_up
make docker_populate
```

Then, create your environment:

```bash
conda env create -f environment.yml
```
and activate it and install the needed packages:

```bash
conda activate quay
pip install -e .[dev]
pip install jupyter
```
Now, you can use the notebooks by starting jupyter-lab in your environment or via VSCode.